# Read in unit info

In [ ]:
#| default_exp parsing

In [15]:
#| exporti
import pandas as pd
import re

In [16]:
#| exporti

def wargear_to_weapon(wg):
    return {
        'name': wg['name'],
        'type': wg['type'].lower(),
        'range': wg['range'],
        'attacks': wg['A'], 
        'bsws': int(str(wg['BS_WS']).strip('+')) if str(wg['BS_WS'])!='nan' else 0,
        'strength': int(wg['S']) if 'D' not in wg['S'] else 7, # TODO: Hack for now as it is 2D6 twice and 6+D6 once
        'AP': int(wg['AP']), 
        'damage': wg['D'],
        'kws': [ kw.strip() for kw in re.split(r',|\.',str(wg['description']))],
        'simplified': 'D' in str(wg['S']) # Warn if something here was simplified 
    }

def model_to_target(m):
    return { 'name': m['name'], 'toughness': int(m['T'].strip('*')), 
            'save': int(m['Sv'].strip('+')), 
            'invuln': int(m['inv_sv'].strip('*')) if str(m['inv_sv'])!='-' else None, 
            'wounds': int(m['W']),
            'simplified': '*' in m['inv_sv'] or '*' in m['T'] # Warn if something here was simplified 
    }

In [19]:
#| export

def csv_files_to_dict(csv_dir):
    files = ['Factions','Datasheets','Datasheets_abilities','Datasheets_keywords','Datasheets_models','Datasheets_wargear','Datasheets_unit_composition','Abilities','Last_update']
    dfs = {}
    for f in files:
        dfs[f] = pd.read_csv(csv_dir+'/'+f+'.csv',sep='|')

    # Remove link tags from wargear descriptions
    dfs['Datasheets_wargear']['description'] = dfs['Datasheets_wargear']['description'].str.replace('<[^>]*>','',regex=True).str.lower()

    # Simplify abilities down to a dict of lists
    ability_dict = pd.Series(dfs['Abilities']['name'].values,index=dfs['Abilities']['id']).to_dict()
    df = dfs['Datasheets_abilities']
    df.loc[~df['ability_id'].isna(),'name'] = df.loc[~df['ability_id'].isna(),'ability_id'].replace(ability_dict)
    df.loc[~df['parameter'].isna(),'name'] += ' ' + df.loc[~df['parameter'].isna(),'parameter']
    df['name'] = df['name'].str.lower()
    abilities = df.groupby('datasheet_id')['name'].apply(list)

    df = dfs['Datasheets_keywords']
    df['keyword'] = df['keyword'].str.lower()
    keywords = df.groupby('datasheet_id')['keyword'].apply(list)

    rd = {}

    dsgb = dfs['Datasheets'].groupby('faction_id')
    dsmgb = dfs['Datasheets_models'].groupby('datasheet_id')
    dswggb = dfs['Datasheets_wargear'].groupby('datasheet_id')
    for i, f in dfs['Factions'].iterrows():

        dsl = { ds['name']:{ 'name': ds['name'], 'id': ds['id'] } for _,ds in dsgb.get_group(f['id']).iterrows() }
        rd[f['name']] = dsl

        for ds in dsl.values():
            if ds['id'] not in dswggb.groups: continue
            ds['weapons'] = list(dswggb.get_group(ds['id']).apply(wargear_to_weapon,axis=1))

            if ds['id'] not in dsmgb.groups: continue
            ds['models'] = list(dsmgb.get_group(ds['id']).apply(model_to_target,axis=1))

            for m in ds['models']:
                m['abilities'] = abilities[ds['id']]
                m['kws'] = keywords[ds['id']]

    return rd
            

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()